<a href="https://colab.research.google.com/github/prithwis/AGI/blob/main/Chandrayaan_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)<br>


![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>




[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

# ChandraYaan

In [1]:
# 1. Update package list and install the NEW opengl names
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb ffmpeg freeglut3-dev python3-opengl libgl1-mesa-dev libglu1-mesa-dev mesa-utils > /dev/null 2>&1

# 2. Install the system dependencies (SWIG is the key here)
#!apt-get update
!apt-get install -y swig build-essential python3-dev > /dev/null 2>&1

# 3. Upgrade pip and setuptools to handle the build process better
!pip install --upgrade pip setuptools wheel > /dev/null 2>&1

# 4. Now install gymnasium with box2d support
!pip install "gymnasium[box2d]" > /dev/null 2>&1

# 5. Install the Python libraries
!pip install pyvirtualdisplay  pygame opencv-python > /dev/null 2>&1


In [2]:
# 1. Install dependencies (Quiet mode)
#!pip install "gymnasium[box2d]" > /dev/null 2>&1

# 2. Verify installation
import gymnasium as gym
try:
    env = gym.make("LunarLander-v3")
    print("🚀 Lunar Lander is ready for takeoff!")
    env.close()
except Exception as e:
    print(f"❌ Installation issue: {e}")
    print("Try: Menu -> Runtime -> Restart Session")

🚀 Lunar Lander is ready for takeoff!


/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [3]:
import pygame
import cv2
import numpy as np
import os
import numpy as np
import random

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import gymnasium as gym
import cv2
import numpy as np
import os

from IPython.display import Video, display


In [10]:
import cv2
import numpy as np
import os

class VideoRecorder:
    def __init__(self, filename='simulation.avi', width=400, height=400, fps=15):
        self.filename = filename
        self.width = width
        # Add 60 pixels to the height for the dedicated dashboard area
        self.dash_height = 60
        self.total_height = height + self.dash_height
        self.fps = fps
        self.fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.video_writer = None

    def start(self):
        if os.path.exists(self.filename):
            os.remove(self.filename)
        # Crucial: The dimensions here MUST match the frames we write later
        self.video_writer = cv2.VideoWriter(
            self.filename, self.fourcc, self.fps, (self.width, self.total_height)
        )

    def record_frame_with_hud(self, frame_array, reward, step):
        # 1. Prepare the environment frame (the 7x7 grid)
        view = cv2.resize(frame_array, (self.width, self.total_height - self.dash_height))
        # Ensure it is in BGR for OpenCV
        #if len(view.shape) == 3 and view.shape[2] == 3:
        #    view = cv2.cvtColor(view, cv2.COLOR_RGB2BGR)

        # 2. Create the light-colored Dashboard (the background for text)
        # BGR (240, 240, 240) is a nice light gray
        dashboard = np.full((self.dash_height, self.width, 3), 240, dtype=np.uint8)

        # 3. Add SMALLER, DARKER text to the dashboard
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.45  # Small and crisp
        thickness = 1
        text_color = (60, 60, 60) # Dark Gray

        cv2.putText(dashboard, f"STEP: {step}", (15, 20),
                    font, font_scale, text_color, thickness, cv2.LINE_AA)
        cv2.putText(dashboard, f"CUMULATIVE REWARD: {reward}", (15, 45),
                    font, font_scale, text_color, thickness, cv2.LINE_AA)

        # 4. Stack them: Dashboard on TOP, Environment on BOTTOM
        combined_frame = np.vstack((dashboard, view))

        # 5. Write to file
        self.video_writer.write(combined_frame)

    def stop(self):
        if self.video_writer:
            self.video_writer.release()

        output_mp4 = self.filename.replace('.avi', '.mp4')
        # Final conversion for browser compatibility
        os.system(f"ffmpeg -y -i {self.filename} -c:v libx264 -pix_fmt yuv420p {output_mp4} -hide_banner -loglevel error")
        return output_mp4

In [5]:
import gymnasium as gym

# Initialize the environment
# 'rgb_array' is required so your VideoRecorder can "see" the frames
env = gym.make("LunarLander-v3", render_mode="rgb_array")

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from collections import deque
import numpy as np

# 1. THE BRAIN (The architecture you used successfully before)
class PowerhouseDQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(PowerhouseDQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x) # Linear output for Q-values


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from collections import deque
import numpy as np


import torch

# Automatically detect if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=20000)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.995
        self.learning_rate = 0.0001

        # 1. Move the Networks to GPU
        self.model = PowerhouseDQN(state_size, action_size).to(device)
        self.target_model = PowerhouseDQN(state_size, action_size).to(device)
        self.update_target_model()

        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def soft_update(self, tau=0.005):
        for target_param, local_param in zip(self.target_model.parameters(), self.model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        # 2. Move input state to GPU
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        with torch.no_grad():
            act_values = self.model(state)
        return torch.argmax(act_values[0]).item()

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return

        minibatch = random.sample(self.memory, batch_size)

        # Convert minibatch to tensors and move to GPU once for speed
        states = torch.FloatTensor(np.array([i[0] for i in minibatch])).to(device)
        actions = torch.LongTensor(np.array([i[1] for i in minibatch])).to(device)
        rewards = torch.FloatTensor(np.array([i[2] for i in minibatch])).to(device)
        next_states = torch.FloatTensor(np.array([i[3] for i in minibatch])).to(device)
        dones = torch.FloatTensor(np.array([i[4] for i in minibatch])).to(device)

        # 3. All calculations now happen on GPU
        # Get Q values for current states
        current_q = self.model(states).gather(1, actions.unsqueeze(1))

        # Get Max Q values for next states from Target Model
        with torch.no_grad():
            next_q = self.target_model(next_states).max(1)[0]
            target_q = rewards + (1 - dones) * self.gamma * next_q

        loss = F.mse_loss(current_q.squeeze(), target_q)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # 1. Update the Target Brain slowly
        self.soft_update(tau=0.005)

        # 2. Decay exploration
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

Using device: cuda


#Tuning to prevent hover

In [8]:
!rm dqn*.*

rm: cannot remove 'dqn*.*': No such file or directory


In [12]:
# 1. Configuration
episodes = 500
MAX_STEPS = 400
batch_size = 128  # Larger batch for the Powerhouse brain
tau = 0.005       # Soft update rate

# Initialize Agent with fresh Epsilon
agentY = DQNAgent(8, 4)
agentY.epsilon = 1.0
agentY.epsilon_decay = 0.995 # Slow exploration

for e in range(episodes):
    state, info = env.reset()
    state = np.reshape(state, [8])
    total_reward = 0

    # --- RECORDER MILESTONES ---
    # Record at start, middle, and end
    should_record = (e == 0 or e == 250 or e == episodes - 1)

    if should_record:
        print(f"🎬 Recording Milestone Episode: {e}")
        # Unique filename for each milestone
        recorder = VideoRecorder(filename=f'lander_clean_ep_{e}.avi', width=600, height=400)
        recorder.start()

    for step_count in range(MAX_STEPS):
        action = agentY.act(state)
        observation, reward, terminated, truncated, info = env.step(action)

        # Pure environment reward - no manual shaping!
        done = terminated or truncated
        next_state = np.reshape(observation, [8])

        agentY.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        # Record frame if milestone
        if should_record:
            frame_bgr = cv2.cvtColor(env.render(), cv2.COLOR_RGB2BGR)
            recorder.record_frame_with_hud(frame_bgr, round(total_reward, 2), step_count)

        if done:
            break

    # --- END OF EPISODE CLEANUP ---
    if should_record:
        recorder.stop()
        print(f"✅ Video for Episode {e} saved.")

    # Training: Replay multiple times to "digest" the Powerhouse brain
    # soft_update is called inside replay
    for _ in range(10):
        agentY.replay(batch_size)

    # Logging
    if e % 25 == 0:
        print(f"Episode: {e}/{episodes} | Score: {total_reward:.2f} | Epsilon: {agentY.epsilon:.2f}")

env.close()

🎬 Recording Milestone Episode: 0
✅ Video for Episode 0 saved.
Episode: 0/500 | Score: -97.77 | Epsilon: 1.00
Episode: 25/500 | Score: -96.19 | Epsilon: 0.29
Episode: 50/500 | Score: -68.20 | Epsilon: 0.08
Episode: 75/500 | Score: -232.67 | Epsilon: 0.05
Episode: 100/500 | Score: -405.32 | Epsilon: 0.05
Episode: 125/500 | Score: -123.95 | Epsilon: 0.05
Episode: 150/500 | Score: -281.76 | Epsilon: 0.05
Episode: 175/500 | Score: -96.77 | Epsilon: 0.05
Episode: 200/500 | Score: -40.91 | Epsilon: 0.05
Episode: 225/500 | Score: -32.30 | Epsilon: 0.05
🎬 Recording Milestone Episode: 250
✅ Video for Episode 250 saved.
Episode: 250/500 | Score: -208.62 | Epsilon: 0.05
Episode: 275/500 | Score: -4.72 | Epsilon: 0.05
Episode: 300/500 | Score: 77.77 | Epsilon: 0.05
Episode: 325/500 | Score: -90.43 | Epsilon: 0.05
Episode: 350/500 | Score: 85.87 | Epsilon: 0.05
Episode: 375/500 | Score: -35.41 | Epsilon: 0.05
Episode: 400/500 | Score: -7.56 | Epsilon: 0.05
Episode: 425/500 | Score: 14.30 | Epsilon: 

In [13]:
#display(Video(first_flight, embed=True))
display(Video("/content/lander_clean_ep_250.mp4", embed=True))
#display(Video(first_flight, embed=True))
display(Video("/content/lander_clean_ep_499.mp4", embed=True))

#Chronobooks <br>
Three science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chrono3) <br>
![alt text](https://blogger.googleusercontent.com/img/a/AVvXsEjsZufX_KYaLwAnJP6bUxvDg5RSPn6r8HIZe749nLWX3RuwyshrYEAUpdw03a9WIWRdnzA9epwJOE05eDJ0Ad7kGyfWiUrC2vNuOskb2jA-e8aOZSx8YqzT8mfZi3E4X1Rz3qlEAiv-aTxlCM976BEeTjx4J64ctY3C_FoV4v9aY_U23F8xRqI5Eg=s1600)